In [ ]:
# Loaded info
material = "concrete"
Lenght = 4000
Height = 200
Width = 200
Thickness = 0

# From Mechanics of Materials 8th and 9th edition by James M 

# Force
Load =  0

# Moment (Source: Cantilever Beam point load at any point Structx)
Distancetofix = 0
MaxMoment = Load * Distancetofix


# Second moment of area in m4
match material:
    case "concrete":
        SecondMomentOfInertia = (Width * (Height^3)) / 12 # concrete
    case "wood":
        SecondMomentOfInertia = (Width * (Height^3)) / 12 # wood 
    case "steel":
        # actual formula here https://amesweb.info/section/i-beam-moment-of-inertia-calculator.aspx
        # Replaced with generic for simplicity for now
        SecondMomentOfInertia = 7763 # cm^4 steel HE 260A


#  Modulus of elasticity in MPa
match material:
    case "concrete":
        ElasticityModulus =  20 # GPA concrete
    case "wood":
        ElasticityModulus =  8 # GPA wood 
    case "steel":
        ElasticityModulus =  210 # GPA steel


# Max deflection (Source: Cantilever Beam point load at any point Structx)
MaxDeflection = (Load * Distancetofix^2) / (6 * ElasticityModulus * SecondMomentOfInertia)

In [6]:
# Formula from: https://structx.com/Stress_Strain_012.html

# ∆T = change in temperature, ᵒK or ᵒC
averagetemp = 12 # C
newmaxtemp = 40 # C
DeltaT = newmaxtemp - averagetemp 

# α = coefficient of thermal expansion, ᵒK-1 or ᵒC-1
material = "concrete"
match material:
# Source for coefficients: https://www.engineeringtoolbox.com/linear-expansion-coefficients-d_95.html
    case "concrete":
        thermalexpension = 9.8 * 10**-6 # concrete
    case "wood":
        thermalexpension = 30 * 10**-6 # wood perpendicular to grain
    case "steel":
        thermalexpension = 12.5 * 10**-6 # steel

# L = length under consideration, in or mm
L = 3600 # mm

# ∆L = change in length, in or mm
DeltaL = thermalexpension * DeltaT * L

print(DeltaL)

0.9878400000000002


In [2]:


# t = duration of impact in seconds
material = "concrete"
match material:
# City of Honolulu building code for impact durations
    case "concrete":
        impacttime = 0.1 # second
    case "wood":
        impacttime = 1 # second 
    case "steel":
        impacttime = 0.5 # second

# Fi = impact force in lb acting at the stillwater level
# w = weight of the object in lb
# V = velocity of water in ft/sec or approximated by 1/2(gds)1/2
# g = gravitational constant (32.2 ft/sec2)
